# University of York's Data Analysis Tool

- Thank you for using our Data Analysis Tool.
- To avoid any issues, please ensure you are using Python version 3.7 to 3.10.
- Ensure the Excel files ACTIVITY_LOG, COMPONENT_CODES, and USER_LOG are located in the same folder as this program.
- Verify that the libraries Tkinter, Pandas, NumPy, Seaborn, and Matplotlib are installed on your computer.
- Happy learning!

In [3]:
import tkinter as tk
from tkinter import messagebox
from tkinter import ttk
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import time

class DataApp:
    def __init__(self, root):
        self.root = root
        self.df = None
        self.df_user = None
        self.df_codes = None
        self.pivot_df = None
        
        # Set the main window
        self.root.title("University of York's Data Analysis Tool")

        # Set background color
        self.root.config(bg="#f0f8ff")

        # Set window size
        self.root.pack_propagate(True)
        self.root.minsize(400, 300)

        # Create a label for the welcome message
        self.welcome_label = tk.Label(root, text="University of York's Data Analysis Tool", font=("Helvetica", 16, "bold"), 
            fg="white", bg="#4CAF50", pady=20)
        self.welcome_label.pack(fill="both")

        # Add some decorative text
        self.decorative_label = tk.Label(root, text="Ready to get started?", font=("Helvetica", 14), fg="black", bg="#f0f8ff")
        self.decorative_label.pack(pady=10)

        # Create 'Load Data' button
        self.load_button = tk.Button(root, text="Load Data", font=("Helvetica", 12), fg="white", bg="#4CAF50",
            command=self.load_data, padx=10, pady=5)
        self.load_button.pack(pady=10)

        # Create 'Close Program' button
        self.close_button = tk.Button(root, text="Close Programme", font=("Helvetica", 12), fg="white", bg="#f44336", 
            command=root.destroy, padx=10, pady=5)
        self.close_button.pack(pady=10)

    def load_data(self):
        try:
            # Load datasets
            self.df = pd.read_csv("ACTIVITY_LOG.csv")
            self.df_user = pd.read_csv("USER_LOG.csv")
            self.df_codes = pd.read_csv("COMPONENT_CODES.csv")

            # Convert to JSON
            self.df.to_json("ACTIVITY_LOG.json", orient="records", indent=4)
            self.df_user.to_json("USER_LOG.json", orient="records", indent=4)
            self.df_codes.to_json("COMPONENT_CODES.json", orient="records", indent=4)

            # Reload from JSON
            self.df = pd.read_json("ACTIVITY_LOG.json")
            self.df_user = pd.read_json("USER_LOG.json", convert_dates=False)
            self.df_codes = pd.read_json("COMPONENT_CODES.json")

            messagebox.showinfo("Success", "Data loaded and converted to JSON successfully!")

            # Update GUI for cleaning options
            self.load_button.pack_forget()
            self.close_button.pack_forget()
            self.welcome_label.config(text="Cleaning Data")
            self.decorative_label.config(text="Data is ready to be cleaned")

            self.clean_button = tk.Button(
                self.root, text="Clean Data", font=("Helvetica", 12), fg="white", bg="#4CAF50", 
                command=self.clean_data, padx=10, pady=5)
            self.clean_button.pack(pady=10)

            self.save_close_button = tk.Button(
                self.root, text="Save & Close Programme", font=("Helvetica", 12), fg="white", bg="#f44336", 
                command=self.save_and_close, padx=10, pady=5)
            self.save_close_button.pack(pady=10)

        except Exception as e:
            messagebox.showerror("Error", f"An error occurred: {e}")
            
    def display_pivot_table(self):
        try:
            # Create the pivot table
            pivot_table = self.pivot_df.pivot_table(
                values=[col for col in self.pivot_df.columns if col.startswith('User_ID')],
                index=['Component', 'YearMonth'],
                aggfunc='sum',
                fill_value=0
            ).reset_index()
            
            # Sort User_ID columns numerically
            user_id_columns = [col for col in pivot_table.columns if col.startswith('User_ID')]
            sorted_user_id_columns = sorted(user_id_columns, key=lambda x: int(x.split('_')[-1]))

            # Create a new window
            table_window = tk.Toplevel()
            table_window.title("The interactions for each user with the Component for each month")

            # Create a Treeview widget
            tree = ttk.Treeview(table_window, show="headings")
            tree["columns"] = list(pivot_table.columns)

            # Set up the headings
            for col in pivot_table.columns:
                tree.heading(col, text=col)
                tree.column(col, width=120, anchor='center')

            # Insert rows in chunks
            for i in range(0, len(pivot_table), 50):  # Load 50 rows at a time
                chunk = pivot_table.iloc[i:i+50]
                for _, row in chunk.iterrows():
                    tree.insert("", "end", values=row.tolist())

            # Pack the Treeview into the window
            tree.pack(fill="both", expand=True)

            # Add a horizontal scrollbar
            h_scrollbar = ttk.Scrollbar(table_window, orient="horizontal", command=tree.xview)
            tree.configure(xscroll=h_scrollbar.set)
            h_scrollbar.pack(side="bottom", fill="x")

            # Add a vertical scrollbar
            v_scrollbar = ttk.Scrollbar(table_window, orient="vertical", command=tree.yview)
            tree.configure(yscroll=v_scrollbar.set)
            v_scrollbar.pack(side="right", fill="y")

        except Exception as e:
            messagebox.showerror("Error", f"Unable to display pivot table: {e}")

    def clean_data(self):
        try:
            if self.df['User Full Name *Anonymized'].isin(self.df_user['User Full Name *Anonymized']).all():
                self.df.insert(0, "Activity_ID", range(len(self.df)))
                self.df_user.insert(0, "Activity_ID", range(len(self.df_user)))
            else:
                messagebox.showerror("Error", "Columns do not match row for row! Data is not ready to be cleaned.")
                return

            # Data cleaning steps
            self.df = self.df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
            self.df_user = self.df_user.applymap(lambda x: x.strip() if isinstance(x, str) else x)
            self.df_user["DateTime"] = pd.to_datetime(self.df_user["Date"] + " " + self.df_user["Time"], dayfirst=True, errors="coerce")
            self.df_user.drop(columns=['Date', 'Time'], inplace=True)

            exclude_component_system_folder = ["System", "Folder"]
            self.df = self.df[~self.df["Component"].isin(exclude_component_system_folder)]
            self.df.rename(columns={"User Full Name *Anonymized": "User_ID"}, inplace=True)
            self.df_user.rename(columns={"User Full Name *Anonymized": "User_ID"}, inplace=True)

            # Correct the spelling of 'attendence' to 'attendance'
            self.df['Component'] = self.df['Component'].replace(r'(?i)attendence', 'attendance', regex=True)
        
            # Handling blanks, nulls, and weird characters
            self.df['Component'] = self.df['Component'].fillna('Unknown').replace(r'[^\w\s]', '', regex=True).str.capitalize()
            self.df['Action'] = self.df['Action'].fillna('Unknown').replace(r'[^\w\s]', '', regex=True).str.capitalize()
            self.df['Target'] = self.df['Target'].fillna('Unknown').replace(r'[^\w\s]', '', regex=True).str.capitalize()
            
            # Merging datasets
            self.df = self.df.merge(self.df_user[["Activity_ID", "DateTime"]], on="Activity_ID", how="left")
            self.df.drop(columns="Activity_ID", inplace=True)
            self.df['YearMonth'] = self.df['DateTime'].dt.to_period('M')
            
            #Reshaping data
            self.pivot_df = self.df.groupby(['YearMonth', 'Component', 'User_ID']).size().unstack(fill_value=0).reset_index()
            self.pivot_df.rename(columns=lambda x: f'User_ID_{x}' if isinstance(x, int) else x, inplace=True)
            self.display_pivot_table()

            messagebox.showinfo("Data Cleaning", "Data cleaned successfully!")

            # Update GUI for analysis options
            self.clean_button.pack_forget()
            self.save_close_button.pack_forget()
            self.welcome_label.config(text="Statistics")
            self.decorative_label.config(text="Calculate mean, mode and median for each component")
            
            self.component_buttons = []
            components = ["Quiz", "Lecture", "Assignment", "Attendance", "Survey"]
            for component in components:
                button = tk.Button(self.root, text=component, font=("Helvetica", 12), fg="white", bg="#4CAF50",
                    command=lambda comp=component: self.mean_mode_median(comp), padx=10, pady=5)
                button.pack(pady=5)
                self.component_buttons.append(button)
 
            self.graphs_button = tk.Button(
                self.root, text="See Graphs", font=("Helvetica", 12), fg="white", bg="#3F8A8C", 
                command=self.graphs, padx=10, pady=5)
            self.graphs_button.pack(pady=10)

            self.save_close_button = tk.Button(
                self.root, text="Save & Close Programme", font=("Helvetica", 12), fg="white", bg="#f44336", 
                command=self.save_and_close, padx=10, pady=5)
            self.save_close_button.pack(pady=10)


        except Exception as e:
            messagebox.showerror("Error", f"An error occurred during data cleaning: {e}")
            
        
    def mean_mode_median(self, selected_component):
        try:
            # Create the pop-up window
            stats_window = tk.Toplevel()
            stats_window.title(f"Statistics Table for {selected_component}")

            # Create a Treeview widget
            tree = ttk.Treeview(stats_window, show="headings")
            tree["columns"] = ["Component", "YearMonth", "Mean", "Mode", "Mode: Occurrences", "Median"]

            # Set up the headings
            for col in tree["columns"]:
                tree.heading(col, text=col)
                tree.column(col, width=150, anchor="center")  # Adjust column width

            # Function to calculate statistics for a specific component and time period
            def get_statistics(df, component, year_month=None):
                if year_month:
                    filtered_df = df[(df["Component"] == component) & (df["YearMonth"] == year_month)]
                else:
                    filtered_df = df[df["Component"] == component]

                if filtered_df.empty:
                    # Return "No data" for the statistics
                    return "No data", "No data", "No data", "No data"

                user_values = filtered_df.iloc[:, 2:].values.flatten()
                mean_value = np.nanmean(user_values)
                value_series = pd.Series(user_values)
                mode_counts = value_series.value_counts()
                mode_value = mode_counts.idxmax()
                occurrences = mode_counts.max()
                median_value = value_series.median()

                return round(mean_value, 1), round(mode_value, 1), occurrences, round(median_value, 1)

            # Get statistics for each YearMonth in pivot_df
            year_months = self.pivot_df['YearMonth'].unique()
            for month in year_months:
                stats = get_statistics(self.pivot_df, selected_component, month)
                formatted_month = str(month)                 
                tree.insert("", "end", values=(selected_component, formatted_month, *stats))

            # Add semester statistics
            semester_stats = get_statistics(self.pivot_df, selected_component)
            tree.insert("", "end", values=(selected_component, "Entire Semester", *semester_stats))

            # Pack the Treeview into the window
            tree.pack(fill="both", expand=True)

            # Add a scrollbar
            scrollbar = ttk.Scrollbar(stats_window, orient="horizontal", command=tree.xview)
            tree.configure(xscroll=scrollbar.set)
            scrollbar.pack(side="bottom", fill="x")

        except Exception as e:
            messagebox.showerror("Error", f"Unable to display statistics: {e}")
        
    
    def graphs(self):
        try:
            # Update GUI for analysis options
            for button in self.component_buttons:
                button.pack_forget()
                
            self.graphs_button.pack_forget()
            self.save_close_button.pack_forget()
            self.welcome_label.config(text="Graphs")
            self.decorative_label.config(text="Visualise the following graphs:")

            # Boxplot button
            self.boxplot_button = tk.Button(
                self.root, text="Boxplot", font=("Helvetica", 12), fg="white", bg="#4CAF50", 
                command=self.boxplot, padx=10, pady=5)
            self.boxplot_button.pack(pady=10)

            # Heatmap button
            self.heatmap_button = tk.Button(
                self.root, text="Heatmap", font=("Helvetica", 12), fg="white", bg="#4CAF50", 
                command=self.heatmap, padx=10, pady=5)
            self.heatmap_button.pack(pady=10)
            
            # Correlation button
            self.correlation_button = tk.Button(
                self.root, text="Correlation", font=("Helvetica", 12), fg="white", bg="#4CAF50", 
                command=self.correlation, padx=10, pady=5)
            self.correlation_button.pack(pady=10)

            # Save & Close button
            self.save_close_button = tk.Button(
                self.root, text="Save & End Programme", font=("Helvetica", 12), fg="white", bg="#f44336", 
                command=self.save_and_close, padx=10, pady=5)
            self.save_close_button.pack(pady=10)

        except Exception as e:
            messagebox.showerror("Error", f"An error occurred during graph visualization setup: {e}")

    def boxplot(self):
        try:
            # Reshape the data for the box plot
            boxplot_df = self.df.groupby(['Component', 'User_ID']).size().unstack(fill_value=0).reset_index()

            boxplot_df = boxplot_df.melt(
                id_vars=["Component"],
                value_vars=boxplot_df.columns[1:],
                var_name="User_ID",
                value_name="Interactions",
            )

            # Filter for the desired components
            boxplot_df = boxplot_df[boxplot_df["Component"].isin(["Assignment", "Quiz", "Lecture", "Book", "Project", "Course"])]

            # Create a new top-level window
            plot_window = tk.Toplevel(self.root)
            plot_window.title("User Interactions Boxplot")

            # Create the box plot
            fig, ax = plt.subplots(figsize=(10, 6))
            sns.boxplot(x="Component", y="Interactions", data=boxplot_df, palette='Set2', ax=ax)
            ax.set_title("Distribution of User Interactions per Component")
            ax.set_xlabel("Component")
            ax.set_ylabel("Number of Interactions")
            ax.set_xticklabels(ax.get_xticklabels(), rotation=45)
            plt.tight_layout()

            # Embed the plot into the Tkinter window using FigureCanvasTkAgg
            canvas = FigureCanvasTkAgg(fig, master=plot_window)  
            canvas.get_tk_widget().pack()  
            canvas.draw()

        except Exception as e:
            messagebox.showerror("Error", f"An error occurred while creating the boxplot: {e}")

    def heatmap(self):
        try:
            # Prepare the heatmap data
            heatmap_df = self.df.groupby(['Component', 'User_ID']).size().unstack(fill_value=0).reset_index()

            # Filter for the desired components
            heatmap_df = heatmap_df[heatmap_df["Component"].isin(["Assignment", "Quiz", "Lecture", "Book", "Project", "Course"])]

            # Set 'Component' as the index and transpose the dataframe to align columns as User_IDs
            heatmap_df = heatmap_df.set_index('Component').transpose()

            # Create a new top-level window
            heatmap_window = tk.Toplevel(self.root)
            heatmap_window.title("User Interactions Heatmap")

            # Create the heatmap plot
            fig, ax = plt.subplots(figsize=(10, 6))
            sns.heatmap(heatmap_df, cmap='YlGnBu', cbar=True, ax=ax)
            ax.set_title('Heatmap: User_ID vs Component')

            # Embed the heatmap into the Tkinter window using FigureCanvasTkAgg
            canvas = FigureCanvasTkAgg(fig, master=heatmap_window)
            canvas.get_tk_widget().pack()
            canvas.draw()
        
        except Exception as e:
            messagebox.showerror("Error", f"An error occurred while creating the heatmap: {e}")

    def correlation(self):
        try:
            # Preparing data
            corr_df = self.df.groupby(['Component', 'User_ID']).size().unstack(fill_value=0).reset_index()

            # Filter the pivot DataFrame
            corr_df = corr_df[corr_df["Component"].isin(["Assignment", "Quiz", "Lecture", "Book", "Project", "Course"])]

            # Encode 'Component' as numerical values
            corr_df['Component_Encoded'] = corr_df['Component'].astype('category').cat.codes
            numeric_data = corr_df.drop(columns=['Component']).set_index('Component_Encoded').transpose()

            # Calculate correlation (Spearman is good for ordinal relationships)
            correlation_matrix = numeric_data.corr(method='spearman')

            # Create a new top-level window using tk.Toplevel() for the correlation matrix
            corr_window = tk.Toplevel(self.root)
            corr_window.title("Correlation between the ‘User_ID’ and ‘Component’")

            # Plot the correlation heatmap
            fig, ax = plt.subplots(figsize=(12, 8))  # Adjust size as needed
            sns.heatmap(correlation_matrix, cmap='coolwarm', annot=True, fmt=".2f", cbar=True, ax=ax)

            # Get the original component names from the DataFrame
            component_names = corr_df['Component'].unique()

            # Set the axis labels with the actual component names
            ax.set_xticklabels(component_names)
            ax.set_yticklabels(component_names)

            # Set the title
            ax.set_title('Correlation: User_ID vs Component')
            ax.set_xlabel('Components')
            ax.set_ylabel('Components')

            # Embed the heatmap into the Tkinter window using FigureCanvasTkAgg
            canvas = FigureCanvasTkAgg(fig, master=corr_window) 
            canvas.get_tk_widget().pack()  
            canvas.draw()

        except Exception as e:
            messagebox.showerror("Error", f"An error occurred while creating the correlation matrix: {e}")
    
    
    def save_and_close(self):
        try:
            # Save the DataFrames as JSON
            if self.df is not None:
                self.df.to_json("ACTIVITY_LOG_cleaned.json", orient="records", indent=4)
            if self.df_user is not None:
                self.df_user.to_json("USER_LOG_cleaned.json", orient="records", indent=4)
            if self.df_codes is not None:
                self.df_codes.to_json("COMPONENT_CODES_cleaned.json", orient="records", indent=4)
            if self.pivot_df is not None:
                self.pivot_df.to_json("pivot_df_cleaned.json", orient="records", indent=4)

            # Show success message
            messagebox.showinfo("Save", "All data saved as JSON successfully!")

            # Quit and destroy the root window
            self.root.destroy()

        except Exception as e:
            messagebox.showerror("Error", f"An error occurred while saving or closing the program: {e}")


# Main program
if __name__ == "__main__":
    root = tk.Tk()
    root.title("Data Processing App")
    app = DataApp(root)
    root.mainloop()